### OpenDART 재무 및 공시 정보 수집
  - 기능: 기업 공시 목록 조회 및 재무제표 수집
  - 대상: 지정된 기업(환경변수 DART_CORP_NAME에서 설정, 기본값: 삼성전자)
  - 수집기간: 2024년 1월 1일부터 현재까지의 공시 목록
  - 데이터소스: 금융감독원 전자공시시스템 (OpenDART)
  - 주요내용:
    - 기업별 공시 목록 조회 (최근 100건)
    - 단일 회사 전체 재무제표 수집 (2023년 사업보고서 기준)
    - CSV 파일로 저장

In [ ]:
import os
import pandas as pd
import requests
from datetime import datetime

os.makedirs('data/collected', exist_ok=True)

# API_KEY = os.environ.get('OPEN_DART_API_KEY')
API_KEY = '코드'
BASE = 'https://opendart.fss.or.kr/api'



In [18]:
def fetch_recent_filings(corp_code: str, bgn_de: str, end_de: str) -> pd.DataFrame:
    url = f"{BASE}/list.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bgn_de': bgn_de,
        'end_de': end_de,
        'page_no': 1,
        'page_count': 100
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    return pd.DataFrame(js.get('list', []))

# 회사 코드 맵은 빠르게 한 번 호출
import io, zipfile, xmltodict

def get_corp_code_map():
    if not API_KEY:
        return {}
    url = f"{BASE}/corpCode.xml"
    params = {'crtfc_key': API_KEY}
    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    z = zipfile.ZipFile(io.BytesIO(r.content))
    name = [n for n in z.namelist() if n.lower().endswith('.xml')][0]
    xml_bytes = z.read(name)
    doc = xmltodict.parse(xml_bytes)
    lists = doc.get('result', {}).get('list', [])
    if isinstance(lists, dict):
        lists = [lists]
    mapping = {}
    for it in lists:
        nm = it.get('corp_name')
        cd = it.get('corp_code')
        if nm and cd:
            mapping[nm] = cd
    return mapping


In [19]:
from datetime import datetime

corp_map = get_corp_code_map() if API_KEY else {}
target_name = os.environ.get('DART_CORP_NAME', '삼성전자')
corp_code = corp_map.get(target_name)

if API_KEY and corp_code:
    today = datetime.today().strftime('%Y%m%d')
    filings = fetch_recent_filings(corp_code, '20240101', today)
else:
    filings = pd.DataFrame()


In [ ]:
filings.head()


In [21]:
path1 = f'data/dart_{corp_code}_filings.csv' if API_KEY and corp_code else None
if API_KEY and corp_code and not filings.empty:
    filings.to_csv(path1, index=False)


In [ ]:
f"Saved: {path1}" if path1 and API_KEY and corp_code and not filings.empty else "No filings saved"


In [ ]:
# 재무제표 단일회사 전체 계정과목
bsns_year = '2023'
reprt_code = '11011'  # 사업보고서
if API_KEY and corp_code:
    url_fs = f"{BASE}/fnlttSinglAcntAll.json"
    params_fs = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': bsns_year,
        'reprt_code': reprt_code,
        'fs_div': 'CFS'    # 'CFS':현금 흐름표
    }
    r_fs = requests.get(url_fs, params=params_fs, timeout=30)
    r_fs.raise_for_status()
    js_fs = r_fs.json()
    fs = pd.DataFrame(js_fs.get('list', []))
else:
    fs = pd.DataFrame()


In [ ]:
fs.head()


In [26]:
path2 = f'data/dart_{corp_code}_fs_2023.csv' if API_KEY and corp_code else None
if API_KEY and corp_code and not fs.empty:
    fs.to_csv(path2, index=False)


In [ ]:
f"Saved: {path2}" if path2 and not fs.empty else "No FS saved"
